# Week 4
* Joins
* Visualizations ( Scatterplots, Histograms, and Distribution Plots,Violin, Box,)
* +String Operators
* Data Cleaning
* More EDA

In [ ]:
# Importing necessary libraries and modules
import plotly.express as px
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display
import os
import geopandas as gpd
from keplergl import KeplerGl
import plotly.figure_factory as ff

In [ ]:
# Import pandas
import pandas as pd

# Create the first DataFrame: employees
employees = pd.DataFrame({
    'Employee_ID': [101, 102, 103, 104],
    'Name': ['Alice', 'Bob', 'Charlie', 'David'],
    'Salary': [3500, 4200, 1200, 830]
})

# Create the second DataFrame: departments
departments = pd.DataFrame({
    'Employee_ID': [101, 102, 104, 105],
    'Department': ['HR', 'Finance', 'IT','Marketing'],
    'Job_Level': ['5', '4', '2', '1']
})
display(employees)
display(departments)


In [ ]:
# Perform an inner join on 'Employee_ID' (default behavior)
merged_data = pd.merge(employees, departments, on='Employee_ID')

# Display the result
merged_data

In [ ]:
# Perform an inner join on 'Employee_ID' (default behavior)
merged_data = pd.merge(employees, departments, on='Employee_ID', how='inner')

# Display the result
merged_data

In [ ]:
# Perform an outer join on 'Employee_ID' , 
merged_data = pd.merge(employees, departments, on='Employee_ID', how='outer')

# Display the result
merged_data

In [ ]:
# Perform an left join on 'Employee_ID'
# In this case you will fetch all employees even if they dont have a department assigned 
merged_data = pd.merge(employees, departments, on='Employee_ID', how='left')

# Display the result
merged_data

In [ ]:
# Perform an right join on 'Employee_ID'
# In this case you will fetch all departments even if there is no employees there 
merged_data = pd.merge(employees, departments, on='Employee_ID', how='right')

# Display the result
merged_data

In [ ]:
px.bar(merged_data, x = "Department", y = "Salary", color = "Job_Level")

In [ ]:
merged_data.rename(columns = {"Employee_ID" : "Employee_No"})

In [ ]:
merged_data.columns = ['Employee_No','Employee_Name','Employee_Salary','Employee_Department','Employee_Level']
merged_data

In [ ]:
#outer join with data cleansing

# Perform a left join to combine employees and departments
combined = pd.merge(employees, departments, on='Employee_ID', how='outer')

# Replace missing numerical values with the median
for column in combined.select_dtypes(include=['float64', 'int64']).columns:
    combined[column].fillna(combined[column].median(), inplace=True)

# Replace missing string values with "Unknown"
for column in combined.select_dtypes(include=['object']).columns:
    combined[column].fillna("Unknown", inplace=True)

# Display the cleaned DataFrame
display(combined)

In [ ]:
#join on multiple columns

# Create the first DataFrame: employees
employees = pd.DataFrame({
    'Employee_ID': [101, 102, 103, 104],
    'Department': ['HR', 'Finance', 'IT', 'IT'],
    'Name': ['Alice', 'Bob', 'Charlie', 'David']
})

# Create the second DataFrame: departments
departments = pd.DataFrame({
    'Employee_ID': [101, 102, 104],
    'Department': ['HR', 'Finance', 'IT'],
    'Job_Level': ['5', '4', '2']
})

# Perform a merge on multiple columns
merged = pd.merge(employees, departments, on=['Employee_ID', 'Department'], how='inner')

# Display the result
print("Merged DataFrame:")
merged


In [ ]:
# append was deprecated do not confuse merge with concat

# DataFrame 1
df1 = pd.DataFrame({'ID': [1, 2, 3], 'Name': ['Alice', 'Bob', 'Charlie']})

# DataFrame 2
df2 = pd.DataFrame({'ID': [4, 5], 'Name': ['David', 'Eve']})

# Append DataFrames using pd.concat
appended_df = pd.concat([df1, df2]).reset_index(drop=True)

appended_df


# Advanced Visualizations

In [ ]:
# Step 1: Load GDP, Life Expectancy, Population, and Continents Datasets
gdp_data = pd.read_csv('gdp.csv')
life_expectancy_data = pd.read_csv('life_expectancy.csv')
population_data = pd.read_csv('population.csv')
continents_data = pd.read_csv('continents.csv') 

# Rename columns for clarity
gdp_data.rename(columns={"GDP (constant 2010 US$)": "GDP"}, inplace=True)
life_expectancy_data.rename(columns={"Life expectancy": "Life_Expectancy"}, inplace=True)
population_data.rename(columns={"Total population (Gapminder, HYDE & UN)": "Population"}, inplace=True)
continents_data.rename(columns={"Country": "Entity"}, inplace=True)  # Match 'Entity' column for merging

# Step 2: Merge the datasets on Entity, Code, and Year
merged_data = pd.merge(gdp_data, life_expectancy_data, on=["Entity", "Code", "Year"], how="inner")
merged_data = pd.merge(merged_data, population_data, on=["Entity", "Code", "Year"], how="inner")
merged_data = pd.merge(merged_data, continents_data, on="Entity", how="left")  # Add continent info

# Display the merged dataset to ensure everything is included
merged_data.head()

In [ ]:
merged_data['GDP']= round(merged_data.GDP/1_000_000,2)
merged_data.sample()

In [ ]:
# === Dynamic Scatter Plot using Plotly === #
# Create a dynamic scatter plot

# Ensure the 'Year' column is numeric and sort the data by 'Year'
merged_data['Year'] = pd.to_numeric(merged_data['Year'], errors='coerce')
merged_data = merged_data.dropna(subset=["Year"])  # Drop rows with invalid years
merged_data["Year"] = merged_data["Year"].astype(int)  # Convert Year to integer
merged_data = merged_data.sort_values(by='Year')

# Define a scaling factor for population marker sizes
size_scale = 1e-6  # Adjust this value to fine-tune the marker sizes

# Create the dynamic scatter plot
fig = px.scatter(
    merged_data,
    x="GDP",
    y="Life_Expectancy",
    animation_frame="Year",
    animation_group="Entity",
    size=merged_data["Population"] * size_scale,  # Scale down population for marker sizes
    color="Entity",
    hover_name="Entity",
    log_x=True,  # GDP is typically better visualized on a log scale
    title="Dynamic Scatter Plot: GDP (Millions) vs Life Expectancy Over Time",
    labels={"GDP": "GDP (Millions, log scale)", "Life_Expectancy": "Life Expectancy"}
)

# Update marker styling (outline color and width)
fig.update_traces(marker=dict(line=dict(width=1, color='DarkSlateGrey')))

# Adjust axis ranges for the scaled GDP
fig.update_layout(
    xaxis=dict(range=[0, 8]),  # Adjust log scale range for GDP (log10(1 to 10 million))
    yaxis=dict(range=[30, 90])  # Adjust Life Expectancy range
)

# Improve the layout and legend
fig.update_layout(
    legend=dict(title="Countries", font=dict(size=10)),
    margin=dict(l=50, r=50, t=50, b=50),
    height=600,  # Adjust plot height
    width=1000,  # Adjust plot width
)

# Display the Plotly figure
fig.show()

In [ ]:
#3D Scatter Plot
#A 3D scatter plot to represent the relationships between GDP, Life Expectancy, and Population.
fig = px.scatter_3d(
    merged_data,
    x="GDP",
    y="Life_Expectancy",
    z="Population",
    color="Entity",
    size="Population",  # Bubble size
    log_x=True,  # Log scale for GDP
    title="3D Scatter Plot: GDP, Life Expectancy, and Population"
)

fig.update_layout(height=700, width=900)
fig.show()

In [ ]:
#GDP and Life Expectancy Timeline (Facet Grid)
#Compare trends across countries using small multiples.
# Filter for a subset of countries
subset_countries = ["United States", "China", "India", "Brazil", "Germany","Mexico"]
subset_data = merged_data[merged_data["Entity"].isin(subset_countries)]

fig = px.line(
    subset_data,
    x="Year",
    y="GDP",
    color="Entity",
    facet_col="Entity",  # Create a facet for each selected country
    facet_col_wrap=3,  # Wrap facets into rows
    title="GDP Trends for Selected Countries Over Time"
)

fig.update_layout(height=800, width=1000)
fig.show()


In [ ]:
#Sunburst Chart
#A hierarchical visualization to group countries by GDP brackets, population, or life expectancy.
# Create GDP brackets
merged_data["GDP_Bracket"] = pd.cut(
    merged_data["GDP"],
    bins=[0, 1e5, 1e6, 1e7, 1e8],
    labels=["Low", "Lower-Middle", "Upper-Middle", "High"]
)

fig = px.sunburst(
    merged_data,
    path=["GDP_Bracket", "Entity"],  # Hierarchical levels: GDP bracket → Country
    values="Population",  # Size of slices based on Population
    color="Life_Expectancy",  # Color based on Life Expectancy
    title="Sunburst Chart: Population by GDP Bracket"
)

fig.update_layout(height=700, width=700)
fig.show()

In [ ]:
#Stacked Area Chart
#Show cumulative GDP, life expectancy, or population trends over time by country.
fig = px.area(
    merged_data,
    x="Year",
    y="GDP",
    color="Entity",
    title="Stacked Area Chart: GDP Trends Over Time by Country",
    labels={"GDP": "GDP (constant 2010 US$)"}
)

fig.update_layout(height=600, width=900)
fig.show()

In [ ]:
#Parallel Coordinates Plot
#Compare multiple attributes (e.g., GDP, Life Expectancy, and Population) across countries.

fig = px.parallel_coordinates(
    merged_data,
    dimensions=["GDP", "Life_Expectancy", "Population"],
    color="Year",
    title="Parallel Coordinates Plot: GDP, Life Expectancy, and Population"
)

fig.update_layout(height=600, width=900)
fig.show()

In [ ]:
#Animated Bar Chart Race
#Show the top countries by GDP over time in a dynamic bar race.
fig = px.bar(
    merged_data.sort_values(by=["Year", "GDP"], ascending=[True, False]),
    x="Entity",
    y="GDP",
    color="Entity",
    animation_frame="Year",
    title="Bar Chart Race: Top Countries by GDP Over Time"
)

fig.update_layout(height=600, width=900)
fig.show()

In [ ]:
#Correlation Heatmap
#Visualize correlations between numeric columns like GDP, Population, and Life Expectancy.

# Calculate the correlation matrix and round it to 2 decimal places
correlation_matrix = merged_data[["GDP", "Life_Expectancy", "Population"]].corr().round(2)

# Create the heatmap with rounded annotations
fig = ff.create_annotated_heatmap(
    z=correlation_matrix.values,
    x=correlation_matrix.columns.tolist(),
    y=correlation_matrix.columns.tolist(),
    annotation_text=correlation_matrix.values.round(2).astype(str),  # Rounded annotations
    colorscale="Viridis"
)

# Add the title and adjust layout
fig.update_layout(
    title="Correlation Heatmap",
    title_x=0.5,  # Center the title
    height=600,
    width=600
)

# Display the heatmap
fig.show()

In [ ]:
#Choropleth Map
#Visualize global GDP, population, or life expectancy as a heatmap on a world map.
fig = px.choropleth(
    merged_data,
    locations="Code",  # Country ISO Code
    color="GDP",  # Metric to color by
    hover_name="Entity",  # Country name on hover
    animation_frame="Year",  # Show changes over time
    title="Choropleth Map: Global GDP Over Time",
    labels={"GDP": "GDP (constant 2010 US$)"}
)

fig.update_geos(projection_type="natural earth")  # Use a natural earth projection
fig.show()


In [ ]:
# Step 1: Load the merged data
# (Assuming `merged_data` is already created with GDP, Life Expectancy, and Population)
display("Columns in merged_data before adding centroids:", merged_data.columns)

# Step 2: Load GeoJSON Data for Centroids
geo_data = gpd.read_file("https://raw.githubusercontent.com/datasets/geo-countries/master/data/countries.geojson")

# Calculate centroids for each country
geo_data["Latitude"] = geo_data["geometry"].centroid.y
geo_data["Longitude"] = geo_data["geometry"].centroid.x

# Rename "ADMIN" to match "Entity" in merged_data
geo_data = geo_data.rename(columns={"ADMIN": "Entity"})

# Extract only necessary columns: Entity, Latitude, Longitude
country_centroids = geo_data[["Entity", "Latitude", "Longitude"]]

# Debugging: Check for missing coordinates
missing_coords = merged_data[~merged_data["Entity"].isin(country_centroids["Entity"])]
print("Rows dropped due to missing coordinates:", missing_coords[["Entity"]].drop_duplicates())

# Step 3: Add missing countries manually (e.g., United States)
missing_rows = pd.DataFrame({
    "Entity": ["United States", "Cote d'Ivoire", "Hong Kong"],
    "Latitude": [37.0902, 7.539989, 22.3964],  # Example coordinates
    "Longitude": [-95.7129, -5.54708, 114.1095]
})

# Concatenate the new rows to country_centroids
country_centroids = pd.concat([country_centroids, missing_rows], ignore_index=True)

# Step 4: Merge centroids with the merged dataset
merged_data_with_coords = pd.merge(merged_data, country_centroids, on="Entity", how="left")

# Check if all rows are included now
print("Entities with missing coordinates after the fix:", 
      merged_data_with_coords[merged_data_with_coords["Latitude"].isna()][["Entity"]])

# Drop rows without coordinates
merged_data_with_coords = merged_data_with_coords.dropna(subset=["Latitude", "Longitude"])

In [ ]:
#Bubble Map
#A geographic visualization with bubble sizes representing population and color intensity representing GDP.

fig = px.scatter_geo(
    merged_data_with_coords,
    lat="Latitude",
    lon="Longitude",
    size="Population",  # Bubble size based on Population
    color="GDP",  # Color intensity based on GDP
    hover_name="Entity",
    animation_frame="Year",
    title="Bubble Map: Population and GDP Over Time"
)

fig.update_layout(height=600, width=900)
fig.show()

In [ ]:
# Step 5: Create Kepler Map
kepler_map = KeplerGl(height=600)

# Add the merged data with coordinates to Kepler
kepler_map.add_data(data=merged_data_with_coords, name="GDP, Life Expectancy, and Population")

# Display the map
kepler_map

# Save the updated map to an HTML file
kepler_map.save_to_html(file_name="kepler_map_with_population_fixed.html")

In [ ]:
#Join Plot
px.scatter(merged_data.query("Year == 2017"), x = "GDP", 
           y = "Life_Expectancy",
           marginal_y = "histogram", marginal_x = "histogram")

In [ ]:
#Join Plot
px.scatter(merged_data.query("Year == 2017"), x = "GDP", 
           y = "Life_Expectancy",
           marginal_y = "box", marginal_x = "box")

In [ ]:
sns.jointplot(data = merged_data.query("Year == 2017"), x = "GDP", 
           y = "Life_Expectancy", kind = "hex")

In [ ]:
px.density_heatmap(merged_data.query("Year == 2017"), x = "GDP", 
           y = "Life_Expectancy", marginal_x = "histogram", marginal_y = "histogram")

In [ ]:
px.box(
    merged_data,
    x="Continent",
    y="Life_Expectancy",color="Continent")

In this way we move from this basic plots to more advanced ones

In [ ]:
px.violin(data_frame=merged_data,x='Life_Expectancy')

In [ ]:
px.histogram(data_frame=merged_data,x='Life_Expectancy')

In [ ]:
px.bar(merged_data.groupby('Entity')[['Life_Expectancy']].mean().sort_values(by='Life_Expectancy',ascending=False))

In [ ]:
px.bar(merged_data[merged_data['Year'] == 2017].sort_values(by="GDP",ascending=False), x = "Entity", y = "GDP",color="Entity")

In [ ]:
px.scatter(data_frame=merged_data[merged_data['Year'] == 2017],x='Life_Expectancy',y='GDP',color='Entity')

Data Cleaning

In [ ]:
#Eliminate Duplicates

# Sample dataset with duplicates
data = {'Name': ['Alice', 'Bob', 'Alice', 'David'],
        'Age': [25, 30, 25, 22],
        'City': ['New York', 'Los Angeles', 'New York', 'Chicago']}
df = pd.DataFrame(data)

print("Original Dataset:")
display(df)

# Remove duplicate rows
df_no_duplicates = df.drop_duplicates()

print("\nDataset after removing duplicates:")
df_no_duplicates

In [ ]:
# Dataset with structural errors
data = {'Category': ['N/A', 'not applicable', 'Cat', 'cat', 'Dog', 'dog'],
        'Value': [1, 2, 3, 4, 5, 6]}
df = pd.DataFrame(data)

print("Original Dataset:")
display(df)

# Normalize categories (e.g., lowercase and unify 'N/A')
df['Category'] = df['Category'].str.lower().replace({'n/a': 'not applicable'})

print("\nDataset after resolving structural errors:")
df

In [ ]:
#Filter Outliers
# Dataset with outliers
data = {'Values': [10, 12, 14, 1000, 15, 20]}
df = pd.DataFrame(data)

print("Original Dataset:")
display(df)

# Calculate IQR for outlier detection
Q1 = df['Values'].quantile(0.25)
Q3 = df['Values'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Filter outliers
df_filtered = df[(df['Values'] >= lower_bound) & (df['Values'] <= upper_bound)]

print("\nDataset after filtering outliers:")
df_filtered

In [ ]:
# Handle Missing Data
# Dataset with missing values
data = {'Name': ['Alice', 'Bob', 'Charlie', None],
        'Age': [25, None, 30, 28],
        'City': ['New York', 'Los Angeles', None, 'Chicago']}
df = pd.DataFrame(data)

print("Original Dataset:")
display(df)

# Handle missing data: Replace with placeholder
df_filled = df.fillna({'Name': 'Unknown', 'Age': df['Age'].mean(), 'City': 'Unknown'})

print("\nDataset after handling missing data:")
df_filled

In [ ]:
# Validate

# Descriptive statistics
print("Descriptive Statistics:")
display(df_filled.describe(include='all'))

# Check for unique categories
print("\nUnique Cities:")
df_filled['City'].unique()

# Lecture 4 - filled out

In [ ]:
import pandas as pd
import seaborn as sns
pd.options.display.max_rows = 10

In [ ]:
gdp = pd.read_csv("gdp.csv")
gdp["gdp"] = gdp["GDP (constant 2010 US$)"] / 1_000_000_000

In [ ]:
import plotly.express as px
px.line(gdp, x = "Year", y = "gdp", color = "Entity")

In [ ]:
pop = pd.read_csv("population.csv")

In [ ]:
pop

In [ ]:
px.line(pop.query("Year > 1900"), x = "Year", y = "Total population (Gapminder, HYDE & UN)", color = "Entity")

In [ ]:
pop = pop.rename(columns = {"Total population (Gapminder, HYDE & UN)" : "population"})

In [ ]:
pop

In [ ]:
gdp2017 = gdp.query("Year == 2017")

In [ ]:
pop2017 = pop.query("Year == 2017")
pop2017

In [ ]:
gdp_and_pop_2017 = pd.merge(left = gdp2017, right = pop2017, left_on = "Entity", right_on = "Entity", how = "left")
gdp_and_pop_2017

In [ ]:
gdp_and_pop_2017["gdp_per_capita"] = gdp_and_pop_2017["GDP (constant 2010 US$)"] / gdp_and_pop_2017["population"]
gdp_and_pop_2017

In [ ]:
px.bar(gdp_and_pop_2017.sort_values("gdp_per_capita"), x = "Entity", y = "gdp_per_capita")

In [ ]:
gdp_and_pop_2017.query('Entity == "Kosovo"')

In [ ]:
set(gdp2017["Entity"]) - set(pop2017["Entity"])

In [ ]:
gdp_and_pop_2017

In [ ]:
gdp_and_pop_2017.sort_values("gdp_per_capita")

In [ ]:
pop

In [ ]:
gdp_and_pop = pd.merge(left = gdp, right = pop, left_on = ["Entity", "Year"], right_on = ["Entity", "Year"], how = "inner")
gdp_and_pop

In [ ]:
gdp_and_pop["gdp_per_capita"] = gdp_and_pop["GDP (constant 2010 US$)"] / gdp_and_pop["population"]

In [ ]:
gdp_and_pop

In [ ]:
px.line(gdp_and_pop, x = "Year", y = "gdp_per_capita", color = "Entity")

In [ ]:
gdp_and_pop_by_entity = gdp_and_pop.set_index("Entity")

In [ ]:
gdp_and_pop_by_entity["gdp_per_cap_ratio"] = gdp_and_pop_by_entity["gdp_per_capita"] / gdp_and_pop_by_entity.query("Year == 1960")["gdp_per_capita"]

In [ ]:
gdp_and_pop = gdp_and_pop_by_entity.reset_index()

In [ ]:
gdp_per_cap_ratio_history = gdp_and_pop.dropna()

In [ ]:
px.line(gdp_per_cap_ratio_history, x = "Year", y = "gdp_per_cap_ratio", color = "Entity")

In [ ]:
gdp_and_pop_by_entity = gdp_per_cap_ratio_history.set_index("Entity")
gdp_and_pop_by_entity["pop_ratio"] = gdp_and_pop_by_entity["population"] / gdp_and_pop_by_entity.query("Year == 1960")["population"]
gdp_and_pop_by_entity["gdp_ratio"] = gdp_and_pop_by_entity["gdp"] / gdp_and_pop_by_entity.query("Year == 1960")["gdp"]
gdp_and_pop = gdp_and_pop_by_entity.reset_index()

In [ ]:
gdp_and_pop

In [ ]:
px.line(gdp_and_pop, x = "Year", y = "pop_ratio", color = "Entity")

In [ ]:
px.scatter(gdp_and_pop.query("Year == 2017"), x = "gdp_per_cap_ratio", y = "pop_ratio", color = "Entity")

In [ ]:
px.scatter(gdp_and_pop.query("Year == 2017"), x = "gdp_per_cap_ratio", y = "pop_ratio", color = "Entity", size = "gdp_ratio")

In [ ]:
#Exercise suggestion: 1960 vs 2017 GDP by using merge

### Adding Life Expectancy Data

In [ ]:
life_exp = pd.read_csv("life_expectancy.csv")
life_exp

In [ ]:
life_exp = life_exp[["Entity", "Year", "Life expectancy"]]

In [ ]:
gdp_pop_le = pd.merge(left = gdp_and_pop, right = life_exp, left_on = ["Entity", "Year"], right_on = ["Entity", "Year"], how = "left")

In [ ]:
gdp_pop_le

In [ ]:
px.scatter(gdp_pop_le.query("Year == 2017"), x = "gdp_per_capita", 
           y = "Life expectancy", color = "Entity")

In [ ]:
px.scatter(gdp_pop_le.query("Year == 2017"), x = "gdp_per_capita", 
           y = "Life expectancy", color = "Entity", log_x = True)

In [ ]:
gdp_pop_le_with_log_gdp = gdp_pop_le.copy()

In [ ]:
import numpy as np
gdp_pop_le_with_log_gdp["log_10_gdp_per_capita"] = np.log10(gdp_pop_le_with_log_gdp["gdp_per_capita"])

In [ ]:
fig = px.scatter(gdp_pop_le_with_log_gdp.query("Year == 2017"), x = "log_10_gdp_per_capita", 
           y = "Life expectancy", trendline = "ols")
fig.show()

### Histograms, KDE Plots, Violin Plots, Box Plots

In [ ]:
px.histogram(gdp_pop_le.query("Year == 2017")["Life expectancy"])
# maybe show line

In [ ]:
sns.displot(gdp_pop_le.query("Year == 2017")["Life expectancy"])

In [ ]:
sns.displot(gdp_pop_le.query("Year == 2017")["Life expectancy"], kde = True)

In [ ]:
sns.displot(gdp_pop_le.query("Year == 2017")["Life expectancy"], kde = True, rug = True)

In [ ]:
px.violin(gdp_pop_le.query("Year == 2017")["Life expectancy"])

In [ ]:
px.violin(gdp_pop_le.query("Year == 2017")["Life expectancy"], points = "all")

In [ ]:
continents = pd.read_csv("continents.csv")
continents

In [ ]:
gdp_pop_le = pd.merge(left = gdp_pop_le, right = continents, left_on = "Entity", 
                      right_on = "Country", how = "left")

In [ ]:
continents = ["Africa",  "Europe"]
px.violin(gdp_pop_le.query("Year == 2017 and Continent in @continents").dropna(), y = "Life expectancy",
          color = "Continent")

In [ ]:
continents = ["Africa",  "Europe"]
px.box(gdp_pop_le.query("Year == 2017 and Continent in @continents").dropna(), y = "Life expectancy",
          color = "Continent")

In [ ]:
px.box(gdp_pop_le.query("Year == 2017").dropna(), y = "Life expectancy",
          color = "Continent")

In [ ]:
px.scatter(gdp_pop_le.query("Year == 2017"), x = "gdp_per_capita", 
           y = "Life expectancy",
           marginal_y = "histogram", marginal_x = "histogram")

In [ ]:
px.scatter(gdp_pop_le.query("Year == 2017"), x = "gdp_per_capita", 
           y = "Life expectancy",
           marginal_y = "box", marginal_x = "box")

In [ ]:
px.density_heatmap(gdp_pop_le.query("Year == 2017"), x = "gdp_per_capita", 
           y = "Life expectancy", marginal_x = "histogram", marginal_y = "histogram")

In [ ]:
sns.jointplot(data = gdp_pop_le.query("Year == 2017"), x = "gdp_per_capita", 
           y = "Life expectancy", kind = "hex")

### String Data

In [ ]:
gdp_pop_le[gdp_pop_le["Entity"].str.contains("in")]

In [ ]:
gdp_pop_le[gdp_pop_le["Entity"].str.startswith("B")]

In [ ]:
gdp_pop_le["Entity"].str.upper()

In [ ]:
gdp_pop_le[gdp_pop_le["Entity"].str.contains(" ")]

### Working With Real World Data (Example)

In [ ]:
df = pd.read_csv("indian_states.csv")
df

In [ ]:
df = df.rename(columns = {"Officiallanguages[39]": "official",
                          "Additional officiallanguages[39]": "other",
                          "Population(census 2011)[citation needed]": "population"})

In [ ]:
df

In [ ]:
urdu_states = df[(df["other"].str.contains("Urdu")) | (df["official"].str.contains("Urdu"))]

In [ ]:
urdu_states

In [ ]:
urdu_states["population"].sum()

In [ ]:
df.dtypes

In [ ]:
#pd.to_numeric(df["population"])

In [ ]:
df["population"] = df["population"].str.replace(",", "")

In [ ]:
df

In [ ]:
#pd.to_numeric(df["population"])

In [ ]:
df["population"] = df["population"].str.replace("[48]", "")

In [ ]:
df

In [ ]:
df["population"] = df["population"].str.replace("\[\]", "")

In [ ]:
df

In [ ]:
pd.to_numeric(df["population"])

In [ ]:
df["population"] = pd.to_numeric(df["population"])

In [ ]:
df

In [ ]:
df.dtypes

In [ ]:
urdu_states = df[(df["other"].str.contains("Urdu")) | (df["official"].str.contains("Urdu"))]
urdu_states["population"].sum()

### Even Larger Datasets

In [ ]:
#https://data.cityofberkeley.info/Public-Safety/Berkeley-PD-Stop-Data-Jan-26-2015-to-Sep-30-2020-/4tbf-3yt8 maybe?

In [ ]:
df = pd.read_csv("housing.csv")
df

In [ ]:
df.columns

In [ ]:
df["YrSold"].value_counts()

In [ ]:
df.columns[df.columns.str.contains("Area")]

In [ ]:
fig = px.scatter(df, x = "GrLivArea", y = "SalePrice")
fig

In [ ]:
px.scatter(df, x = "GrLivArea", y = "SalePrice", color = "OverallQual")

In [ ]:
px.scatter(df, x = "GrLivArea", y = "SalePrice", color = "OverallQual",size='LotArea')

In [ ]:
fig = px.scatter(df, x = "GrLivArea", y = "SalePrice", trendline = "ols",
          trendline_color_override="black")
fig

In [ ]:
df

In [ ]:
fig = px.scatter(df, x = "GrLivArea", y = "SalePrice", trendline = "lowess",
          trendline_color_override="black")
fig

# coding_activity_4.1

### Codio Activity 4.1: Basic Joins on Datasets

**Expected Time: 60 Minutes**

**Total Points: 10 Points**

This activity focuses on using the `merge` function to join DataFrame's as seen in Videos 4.1 and 4.2.  Merging DataFrames allows the combination of data along a shared categorical column.  The DataFrames do not need to be the same shape and depending on certain arguments used, the result of a merge can contain different values.  Specifically, using different kinds of joins -- `inner, outer, left, right`, different DataFrames are produced.  

## Index:

- [Problem 1](#Problem-1)
- [Problem 2](#Problem-2)
- [Problem 3](#Problem-3)
- [Problem 4](#Problem-4)
- [Problem 5](#Problem-5)

### The Data: Antarctic Weather Stations

The dataset for this activity is four `.csv` files.  The file names and descriptions of the data are as follows:

- `person.csv`: people who took readings.
- `site.csv`: locations where readings were taken.
- `visited.csv`: when readings were taken at specific sites.
- `survey.csv`: the actual readings. The field quant is short for quantity and indicates what is being measured. Values are rad, sal, and temp referring to ‘radiation’, ‘salinity’ and ‘temperature’, respectively.

Each dataset is loaded below, and basic info displayed.

In [ ]:
site = pd.read_csv('site.csv')

In [ ]:
site.head(2)

In [ ]:
site.info()

In [ ]:
visited = pd.read_csv('visited.csv')

In [ ]:
visited.head(2)

In [ ]:
visited.info()

In [ ]:
person = pd.read_csv('person.csv')
person.head(2)

In [ ]:
survey = pd.read_csv('survey.csv')
survey.head(2)

In [ ]:
survey.info()

[Back to top](#Index:) 

### Problem 1

#### Merging site and visited

**5 Points**

Note that in the `site` data the `name` column is the same information as in the `visited` data `name` column.  Use  `pd.merge` to merge the `site` DataFame with the `visited` one based on the shared column.  Use `site` as your left DataFrame and `visited` as your right.  Assign your result as a DataFrame to `site_visits_df` below.

In [ ]:
### GRADED

site_visits_df = None

### BEGIN SOLUTION
site_visits_df = pd.merge(site, visited, left_on='name', right_on='site')
type(site_visits_df)
### END SOLUTION

# Answer check
print(type(site_visits_df), site_visits_df.shape)
site_visits_df.head()


[Back to top](#Index:) 

### Problem 2

#### Revisit with shared Column Name

**5 Points**


Use the `rename` function to rename the `visited` `site` column to `name` and assign it to the variable `visited_renamed`.

Merge the `site` and the `visited_renamed` datafames on `name` and assign the result of the merge to `site_visits_df2` below.  

Note that your solution should have one column fewer than the solution to Problem 1.

In [ ]:
### GRADED

visited_renamed = None
site_visits_df2 = None

### BEGIN SOLUTION
visited_renamed = visited.rename({'site': 'name'}, axis = 1)
site_visits_df2 = pd.merge(site, visited_renamed, on = 'name')
### END SOLUTION

# Answer check
print(site_visits_df2.shape)
site_visits_df2.head()

[Back to top](#Index:) 

### Problem 3

#### Merging the Survey Results

**5 Points**

The effect of merging the `site` and `visited` is that a DataFrame with each sites name, location, visit id, and date has been created. 

To include the data collected at each site in these visits, the survey data needs to be consulted.  

Identify the shared column in the `site_visits_df2` data and the `survey` data, and merge the data based on this column.  

Rename the column in the `survey` frame from `taken` to `id` not to have a reduntant column in the data and assign your merged DataFrame to `survey_site_visits` below.

In [ ]:
site_visits_df2.head(2)

In [ ]:
survey.head(2)

In [ ]:
### GRADED

survey_site_visits = None

### BEGIN SOLUTION
survey_site_visits = pd.merge(survey.rename({'taken': 'id'}, axis = 1), 
                              site_visits_df2, on = 'id')
### END SOLUTION

# Answer check
display(survey_site_visits)
print(type(survey_site_visits))

[Back to top](#Index:) 

### Problem 4

### Adding Full Names

**5 Points**

Use the `rename` function to rename the `survey_site_visits` `person` column to `person_id` and assign it to the variable `left`.

Use the `rename` function to rename the `person` `id` column to `person_id` and assign it to the variable `right`.

Merge the `left` and the `right` datafames on `person_id` and assign the result of the merge to `full_name_df`. below.  


In [ ]:
### GRADED

full_name_df = None

### BEGIN SOLUTION
left = survey_site_visits.rename({'person': 'person_id'}, axis = 1)
right = person.rename({'id': 'person_id'}, axis = 1)
full_name_df = pd.merge(left, right, on = 'person_id')
### END SOLUTION

# Answer check
print(type(full_name_df))
full_name_df.head(2)

[Back to top](#Index:) 

### Problem 5

#### `left` vs. `right` merge

**5 Points**

Below two new DataFrames -- `df1` and `df2` -- are created with the shared column `name`.  Note the different unique values in the `name` column in each.  Use `pd.merge` and choose the appropriate argument for `how` as either `left` or `right` to create the following table:

<table border="1" class="dataframe">  <thead>    <tr style="text-align: right;">      <th></th>      <th>name</th>      <th>age</th>      <th>member</th>    </tr>  </thead>  <tbody>    <tr>      <th>0</th>      <td>amy</td>      <td>32</td>      <td>True</td>    </tr>    <tr>      <th>1</th>      <td>ben</td>      <td>30</td>      <td>NaN</td>    </tr>    <tr>      <th>2</th>      <td>carlos</td>      <td>40</td>      <td>True</td>    </tr>  </tbody></table>

Assign your results to `ans5` below.

In [ ]:
df1 = pd.DataFrame({'name': ['amy', 'ben', 'carlos'],
                   'age': [32, 30, 40]})
df2 = pd.DataFrame({'name': ['amy', 'carlos', 'lenny'],
                   'member': [True, True, False]})

In [ ]:
### GRADED

ans5 = None

### BEGIN SOLUTION
ans5 = pd.merge(df1, df2, how = 'left', on = 'name')
### END SOLUTION

# Answer check
print(type(ans5))
ans5.head(2)

# coding_activity_4.2

### Codio Activity 4.2: Complex Joins on Datasets

**Expected Time: 60 Minutes**

**Total Points: 10**

![](images/kiva.png)

This assignment focuses on using pandas `merge` to answer questions using multiple data sources.  Here you will combine data and use many of the earlier `pandas` methods together with the `merge` function to extract insights from our data source.  The data comes from the Kiva loan provider organization and was part of a kaggle Data Science for good contest [here](https://www.kaggle.com/kiva/data-science-for-good-kiva-crowdfunding).

#### Index:

- [Problem 1](#Problem-1)
- [Problem 2](#Problem-2)
- [Problem 3](#Problem-3)
- [Problem 4](#Problem-4)
- [Problem 5](#Problem-5)

### The Data

Kiva allows users to fund loans to small organizations around the world.  The four datasets contain a subset of the data provided on kaggle and describe information on the individual loan, when it was given, where the borrowers were located, and what the loans were being used for.  Below, the data is loaded into four DataFrames and the first two rows of each displayed.  Note the shared `id` column across all four datasets.

In [ ]:
import pandas as pd

In [ ]:
crowdsource = pd.read_csv('crowdsource.csv')
crowdsource.head(2)

In [ ]:
demographics = pd.read_csv('demographics.csv')
demographics.head(2)

In [ ]:
financials = pd.read_csv('financials.csv')
financials.head(2)

In [ ]:
use = pd.read_csv('use.csv')
use.head(2)

[Back to top](#Index:) 

### Problem 1

#### Kenyan Loan Amounts

**5 Points**


Use the `demographics` and `financials` data to determine the average loan amount for the country `Kenya`.  

Note that the average loan amount in each country can be calculated by computing the mean of the `funded_amount` column.


Save your response as a float to `ans1` below.

In [ ]:
### GRADED

ans1 = None

### BEGIN SOLUTION
ans1 = pd.merge(demographics.loc[demographics['country'] == 'Kenya'], financials, on = 'id')[['funded_amount']].mean().values[0]
type(ans1)
### END SOLUTION

# Answer check
print(ans1)
print(type(ans1))

[Back to top](#Index:) 

### Problem 2

### El Salvador Top Sector

**5 Points**

Use the `demographics` and `use` datasets to determine the sector of work that received the most loans in `El Salvador`.  

Note that the  the sector of work in each country can be calculated by using `value_counts()` on the `sector` column.


Assign your result as a string to `ans2` below.

In [ ]:
### GRADED

ans2 = None

### BEGIN SOLUTION
ans2 = pd.merge(demographics.loc[demographics['country'] == 'El Salvador'], use, on = 'id')['sector'].value_counts().index[0]
type(ans2)
### END SOLUTION

# Answer check
display(ans2)
print(type(ans2))

### Problem 3

#### Pakistan Loans in Agriculture

**5 Points**


Merge the DataFrames `use` and `demographics` on `id`. Assign your result to `p1`.

Merge the DataFrames `p1` and `financials` on `id`. Assign your result to `a`.

Use `loc` on the `a` DataFrame to select the rows in which `country` is equal to `Pakistan`. Assogn your result to `b`.


Determine the total amount of loans (`funded_amount`) for `Agriculture` in Pakistan.  Assign your results as a float to `ans3` below.

In [ ]:
### GRADED

ans3 = None

### BEGIN SOLUTION
p1 = pd.merge(use, demographics, on = 'id')
a = pd.merge(p1, financials, on = 'id')
b = a.loc[a['country'] == 'Pakistan']
ans3 = b.loc[b['sector'] == 'Agriculture'][['funded_amount']].sum().values[0]

### END SOLUTION

# Answer check
print(ans3)
print(type(ans3))

[Back to top](#Index:) 

### Problem 4

#### Top Total Loan Sector

**5 Points**

Merge the `financials` and `use` DataFrames on `id`. To this, chain a `groupby()` operation on `sector` and use a double square bracket notation to select the column `funded_amount`.



What sector received the most total dollars in funding?  Assign your response as a string to `ans4` below.

In [ ]:
### GRADED

ans4 = None

### BEGIN SOLUTION
ans4 = pd.merge(financials, use, on = 'id').groupby('sector')[['funded_amount']].sum().sort_values(by = 'funded_amount', ascending = False).index[0]
### END SOLUTION

# Answer check
print(ans4)
print(type(ans4))

[Back to top](#Index:) 

### Problem 5

#### Top Loan by Lender Amount

**5 Points**

Merge the DataFrames `financials` and `use` on `id`. Assign your result to `a`.

Merge the DataFrames `a` and `crowdsource` on `id`. Assign your result to `b`.

In the `b` DataFrame, create a new column `ratio`. To this column assign the ratio of the columns `funded_amount` and `lender_count` of the `b` DataFrame.


Determine which loan sector has the highest ratio of currency to lender amount. Assign the sector with the highest ratio as a string to `ans5` below. 

In [ ]:
### GRADED

ans5 = None

### BEGIN SOLUTION
a = pd.merge(financials, use, on = 'id')
b = pd.merge(a, crowdsource, on = 'id')
b['ratio'] = b['funded_amount']/b['lender_count']
ans5 = b.groupby('sector')['ratio'].max().idxmax()
### END SOLUTION

# Answer check
print(ans5)
print(type(ans5))

# coding_activity_4_3

### Codio Activity 4.3: Creating Scatterplots, Histograms, and Distribution Plots

**Expected Time: 60 Minutes**

**Total Points: 12**

This activity focuses on using scatterplots to compare GDP and Life Expectancy.  The built in gapminder data offers GDP and Life Expectancy data similar to the videos, and you will use this to produce a scatterplot that contains more than two dimensions of data and an important transformation.


## Index:

- [Problem 1](#Problem-1)
- [Problem 2](#Problem-2)
- [Problem 3](#Problem-3)
- [Problem 4](#Problem-4)
- [Problem 5](#Problem-5)
- [Problem 6](#Problem-6)

In [ ]:
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd

In [ ]:
gapminder = px.data.gapminder()

In [ ]:
gapminder.head()

[Back to top](#Index:) 

### Problem 1

#### Scatter Plot of GDP vs. Life Expectancy


Use `plotly` to create a scatter plot of the gapminder data with GDP on the $x$-axis and Life Expectancy on the $y$ axis.  

Use the `plt.savefig()` method to save the image as `img1.png` in the `images` folder.

In [ ]:
### GRADED

### BEGIN SOLUTION
px.scatter(data_frame=gapminder, x = 'gdpPercap', y = 'lifeExp')

### END SOLUTION

[Back to top](#Index:) 

### Problem 2

#### Scatter Plot of GDP vs. Life Expectancy with Country

Use `plotly` to create a scatter plot of the gapminder data with GDP on the $x$-axis and Life Expectancy on the $y$ axis and add color to each point based on the `country` column.  

Use the `plt.savefig()` method to save the image as `img2.png` in the `images` folder.

In [ ]:
### GRADED

### BEGIN SOLUTION
px.scatter(data_frame=gapminder, x = 'gdpPercap', y = 'lifeExp', color = 'country')

### END SOLUTION


[Back to top](#Index:) 

### Problem 3

#### Incorporating Population

Use `plotly` to create a scatter plot of the gapminder data with GDP on the $x$-axis and Life Expectancy on the $y$ axis, add color to each point based on the `country` column and size the points based on the population column.  

Use the `plt.savefig()` method to save the image as `img3.png` in the `images` folder.

In [ ]:
### GRADED

### BEGIN SOLUTION
px.scatter(data_frame=gapminder, x = 'gdpPercap', y = 'lifeExp', color = 'country', size = 'pop')

### END SOLUTION

[Back to top](#Index:) 

### Problem 4

#### Logarithmic Transformation

Use `plotly` to create a scatter plot of the gapminder data with GDP on the $x$-axis and Life Expectancy on the $y$ axis, add color to each point based on the `country` column and size the points based on the population column.    Transform the $x$-axis using the `log_x` argument in the scatter function.  

Use the `plt.savefig()` method to save the image as `img3.png` in the `images` folder.

In [ ]:
### GRADED

### BEGIN SOLUTION
px.scatter(data_frame=gapminder, x = 'gdpPercap', y = 'lifeExp', color = 'country', size = 'pop', log_x=True)

### END SOLUTION

[Back to top](#Index:) 

### Problem 5

#### Limit to the year 2007

Limit the data to the year 2007 and create a scatterplot of GDP vs. Life Expectancy, color by country, size the points by population and use a logarithmic transform on the x-axis. 

Use the `plt.savefig()` method to save the image as `img5.png` in the `images` folder.

In [ ]:
### GRADED

### BEGIN SOLUTION
px.scatter(gapminder.query('year == 2007'), x = 'gdpPercap', y = 'lifeExp', color = 'country', size = 'pop',
          log_x = True)

### END SOLUTION

[Back to top](#Index:) 

### Problem 6

#### Boxplots for GDP by Continent

Use `plotly` to create boxplots for GDP by continent.  

Use the `plt.savefig()` method to save the image as `img5.png` in the `images` folder.

In [ ]:
### GRADED

### BEGIN SOLUTION
px.box(data_frame=gapminder, x = 'gdpPercap', color = 'continent')

### END SOLUTION

# coding_activity_4_4

### Codio Activity 4.4: Creating Violin, Box, and Joint Plots

**Expected Time: 60 Minutes**

**Total Points: 20**

This activity focuses on producing the plots demonstrated in Video 4.6 and Video 4.7.  Specifically, histograms, Kernel Density Estimate plots, Violin, and Box plots.  The dataset is the `gapminder` data from plotly.  


#### Index:

- [Problem 1](#Problem-1)
- [Problem 2](#Problem-2)
- [Problem 3](#Problem-3)
- [Problem 4](#Problem-4)
- [Problem 5](#Problem-5)
- [Problem 6](#Problem-6)
- [Problem 7](#Problem-7)
- [Problem 8](#Problem-8)
- [Problem 9](#Problem-9)
- [Problem 10](#Problem-10)


In [ ]:
import pandas as pd
import plotly.express as px
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
gapminder = px.data.gapminder()

[Back to top](#Index:) 

### Problem 1

#### Histogram of Life Expectancy with `plotly`

**1 point**

Create a histogram of the Life Expectancy data using `plotly`.

Use the `plt.savefig()` method to save the image as `img1.png` in the `images` folder.

In [ ]:
### GRADED
#make a histogram of life expectancy in plotly

### BEGIN SOLUTION
hist = px.histogram(gapminder, x= 'lifeExp')
hist
### END SOLUTION

[Back to top](#Index:) 

### Problem 2

#### Histogram of Life Expectancy with `seaborn`

**2 points**

Create a histogram of the Life Expectancy data using `seaborn`. 

Use the `plt.savefig()` method to save the image as `img2.png` in the `images` folder.

In [ ]:
### GRADED
#make a histogram in seaborn

### BEGIN SOLUTION
sns.histplot(data=gapminder, x="lifeExp")

### END SOLUTION

[Back to top](#Index:) 

### Problem 3

#### Histogram and KDE of Life Expectancy with `seaborn` 

**2 Points**

Create a histogram with the KDE of the Life Expectancy data using `seaborn`.

Use the `plt.savefig()` method to save the image as `img3.png` in the `images` folder.

In [ ]:
### GRADED
#make a kde plot in seaborn

### BEGIN SOLUTION
sns.displot(data=gapminder, x="lifeExp", kde=True)

### END SOLUTION

[Back to top](#Index:) 

### Problem 4

#### Histogram, KDE, and rugplot of Life Expectancy with `seaborn`

**2 points**

Create a histogram and KDE of the Life Expectancy data along with the rugplot using `seaborn`.

Use the `plt.savefig()` method to save the image as `img4.png` in the `images` folder.

In [ ]:
### GRADED
#make a rug plot with seaborn

### BEGIN SOLUTION
sns.displot(data=gapminder, x="lifeExp", kde=True)
sns.rugplot(data=gapminder, x="lifeExp")

### END SOLUTION

[Back to top](#Index:) 

### Problem 5

#### Violin Plot of Life Expectancy with `plotly`

**1 point**

Create a Violin Plot of the Life Expectancy data using `plotly`. 

Use the `plt.savefig()` method to save the image as `img5.png` in the `images` folder.

In [ ]:
### GRADED
#create a violin plot with plotly

### BEGIN SOLUTION
fig = px.violin(gapminder, y='lifeExp')
fig
### END SOLUTION

[Back to top](#Index:) 

### Problem 6

#### Violin Plot with points of Life Expectancy with `plotly`

**1 point**

Create a Violin Plot of the Life Expectancy data alongside the actual points using `plotly`. 

Use the `plt.savefig()` method to save the image as `img6.png` in the `images` folder.

In [ ]:
### GRADED
#violin in plotly with points = "all"

### BEGIN SOLUTION
fig = px.violin(gapminder, y='lifeExp',points = 'all')
fig
#plt.savefig('images/img6.png')
### END SOLUTION

[Back to top](#Index:) 

### Problem 7

#### Violin Plot of Life Expectancy by Continent with `plotly`

**1 point**

Create a Violin Plot of the Life Expectancy by continent using `plotly`.

Use the `plt.savefig()` method to save the image as `img7.png` in the `images` folder.

In [ ]:
### GRADED
#violin plot by continent in plotly

### BEGIN SOLUTION
fig = px.violin(gapminder, x='continent',y='lifeExp')
fig
#plt.savefig('images/img7.png')
### END SOLUTION

[Back to top](#Index:) 

### Problem 8

#### Violin Plot of Life Expectancy by Continent with `seaborn`

**2 Points**

Create a Violin Plot of the Life Expectancy by continent using `seaborn`. Note that you will set your `x` argument to continent and `y` to lifeExp.

Use the `plt.savefig()` method to save the image as `img8.png` in the `images` folder.

In [ ]:
### GRADED
#violin plot by continent in seaborn

### BEGIN SOLUTION
sns.violinplot(x="continent", y="lifeExp", data=gapminder)

#plt.savefig('images/img8.png')
### END SOLUTION

[Back to top](#Index:) 

### Problem 9

#### Box Plot of Life Expectancy by Continent with `plotly`

**2 points**

Create a Box Plot of the Life Expectancy by continent using `plotly`. 

Use the `plt.savefig()` method to save the image as `img9.png` in the `images` folder.

In [ ]:
### GRADED
#boxplot by continent with plotly with y = lifeExp

### BEGIN SOLUTION
fig = px.box(gapminder,x="continent", y="lifeExp")
fig
#plt.savefig('images/img9.png')
### END SOLUTION

[Back to top](#Index:) 

### Problem 10

#### Box Plot of Life Expectancy by Continent with `seaborn`

**2 Points**

Create a Box Plot of the Life Expectancy by continent using `seaborn`. Note that you will set your `x` argument to continent and `y` to lifeExp. 

Use the `plt.savefig()` method to save the image as `img10.png` in the `images` folder.

In [ ]:
### GRADED
#seaborn boxplot by continent

### BEGIN SOLUTION
sns.boxplot(x="continent", y="lifeExp", data=gapminder)

#plt.savefig('images/img10.png')
### END SOLUTION

# coding_activity_4.5

### Codio Activity 4.5: String Operations


**Expected Time: 60 Minutes**

**Total Points: 20**


For this activity, you will focus on using string methods on `pandas` series.  Following the examples from video 4.8, a Wikipedia page on the states of the russian federation will be the dataset.  Below is a map of the boundaries for the states. 

#### Index:

- [Problem 1](#Problem-1)
- [Problem 2](#Problem-2)
- [Problem 3](#Problem-3)
- [Problem 4](#Problem-4)

![](https://upload.wikimedia.org/wikipedia/commons/thumb/7/7d/Map_of_federal_subjects_of_Russia_%282014%29.svg/1380px-Map_of_federal_subjects_of_Russia_%282014%29.svg.png)

### Getting the Data

Below, we access the data directly using the url and the `read_html` method from `pandas`.  This method reads any table from a web url in as a list of dataframes.  The data we are interested in is located in the fifth table on the page so we index the list of data accordingly below.

In [ ]:
import pandas as pd

In [ ]:
russian_states = pd.read_csv('russian_states.csv', index_col = 0)

In [ ]:
russian_states.head()

[Back to top](#Index:) 

### Problem 1

#### Using the `contains` method

**5 Points**

Use the string method `contains` to subset the data based on entries in the `Economic region` column containing `Siberian`.  Assign your response as a DataFrame to `ans1` below.

In [ ]:
### GRADED

ans1 = None

### BEGIN SOLUTION
ans1 = russian_states.loc[russian_states['Economic region'].str.contains("Siberian")]
type(ans1)
### END SOLUTION

# Answer check
print(ans1.shape)
ans1.head()

[Back to top](#Index:) 

### Problem 2

#### Using the `startswith` method

**5 Points**

Subset the data based on entries in the `Economic region` column that start with `North`.  Assign your answer as a DataFrame to `ans2` below.

In [ ]:
### GRADED

ans2 = None

### BEGIN SOLUTION
ans2 = russian_states.loc[russian_states['Economic region'].str.startswith('North')]
type(ans2)
### END SOLUTION

# Answer check
print(ans2.shape)
ans2.head()

[Back to top](#Index:) 

### Problem 3: Using the `upper` method

**5 Points**

Use the `upper` method to create a series where the entries in the `Federal district` column all uppercased.  Assign your response as a Series to `ans3` below.

In [ ]:
### GRADED

ans3 = None

### BEGIN SOLUTION
ans3 = russian_states['Federal district'].str.upper()
type(ans3)
### END SOLUTION

# Answer check
print(type(ans3))
ans3.head()

[Back to top](#Index:) 

### Problem 4

#### Examining the Population

**5 Points**
 

Much like the example in the videos, the `Population[17]` column contains problematic characters that need to be replaced before the column can be converted to a float datatype.  Replace the `\[22\]`, `\[23\]`,and `,` values with empty strings. Finally, convert the `Population[17]` column to `float` datatypes.  

Assign the your response as a series to `ans4` below.  

In [ ]:
### GRADED

ans4 = None

### BEGIN SOLUTION
#ans4 = russian_states['Population[17]'].str.replace("\[22\]", " ").str.replace("\[23\]", " ").str.replace(',', '').astype('float')
#type(ans4)
### END SOLUTION

# Answer check
#print(type(ans4))
#ans4.head()

russian_states['Population[17]'] = (
    russian_states['Population[17]']
    .str.replace(r"\[.*?\]", "", regex=True)  # Remove bracketed text using regex
    .str.replace(",", "")  # Remove commas
    .astype(float)  # Convert to float
)

# Check the updated DataFrame
russian_states['Population[17]'].head()

# coding_activity_4_6

### Codio Activity 4.6: Data Cleaning

**Expected Time: 120 Minutes**

**Total Points: 40**

This assignment focuses on making decisions with a somewhat dirty dataset.  From datatype conversion, to object removal and cleaning, to missing value handling, cleaning data will be a regular activity while working with data in the wild.  The dataset comes from kaggle [here](https://www.kaggle.com/shansh8/companies-info-wikipedia-2021), and represents information on businesses pulled from [dbpedia](http://dbpedia.org) pertaining to business indicators from a variety of international companies.


#### Index:

- [Problem 1](#Problem-1)
- [Problem 2](#Problem-2)
- [Problem 3](#Problem-3)
- [Problem 4](#Problem-4)
- [Problem 5](#Problem-5)
- [Problem 6](#Problem-6)
- [Problem 7](#Problem-7)
- [Problem 8](#Problem-8)
- [Problem 9](#Problem-9)


![](images/dbpedia.png)

### Load and Explore the Data

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

In [ ]:
#df = pd.read_csv('data/biz.zip', compression = 'zip')

In [ ]:
#df.head()

In [ ]:
#df.info()

[Back to top](#Index:) 

### Problem 1

#### Plot of Missing Data

**0 Points**

Create a barplot using `matplotlib` with the $x$-axis representing the columns and the $y$-axis representing the count of missing values. Sort these values from least to greatest.  Save your plot in the `results` folder as `missing_plot.png`.  

In [ ]:
### GRADED

### BEGIN SOLUTION
#df.isnull().sum().sort_values().plot(kind = 'bar')
#plt.savefig('results/missing_plot_.png')
#plt.show()
### END SOLUTION

[Back to top](#Index:) 

### Problem 2

#### `name` column

**5 Points**

The column `name` has 31 missing values.

Subset the data to examine these observations where `name` is missing. Assign the result as a DataFrame to `ans2` below.  

In [ ]:
### GRADED

#ans2 = None

### BEGIN SOLUTION
#ans2 = df.loc[df['name'].isnull()]
### END SOLUTION

# Answer check
#print(type(ans2))
#ans2.head()

[Back to top](#Index:) 

### Problem 3

### Filling Missing Values in `name`

**5 Points**

Examining the rows missing values in names shows that in the adjacent `a` column, the end of the url contains what can stand in as a name.  Using the `split` string method on the column `a` to split the URL by `/`.  

Assign your split url's as a series to `ans3` below.

This series will be a collection of lists:

```python
0       [http:, , dbpedia.org, resource, Deutsche_Euro...
1       [http:, , dbpedia.org, resource, Deutsche_Euro...
2       [http:, , dbpedia.org, resource, Industry_of_M...
3       [http:, , dbpedia.org, resource, Industry_of_M...
4       [http:, , dbpedia.org, resource, Industry_of_M...
         .
         .
         .
```



In [ ]:
### GRADED

#ans3 = None

### BEGIN SOLUTION
#ans3 = df['a'].str.split('/')
### END SOLUTION

# Answer check
#print(type(ans3))
#ans3.head()

[Back to top](#Index:) 

### Problem 4

#### Extracting the names

**5 Points**

To extract the last element of this list, use the `.apply` method together with an appropriate `lambda` function to create a series based on the last entry of the above list in each row.  Your results should begin with:

```python
0                        Deutsche_EuroShop
1                        Deutsche_EuroShop
2       Industry_of_Machinery_and_Tractors
3       Industry_of_Machinery_and_Tractors
4       Industry_of_Machinery_and_Tractors
                       ...                
```

Save this series to `ans4` below.

In [ ]:
### GRADED

#ans4 = None

### BEGIN SOLUTION
#ans4 = df['a'].str.split('/').apply(lambda x: x[-1])
### END SOLUTION

# Answer check
#print(type(ans4))
#ans4.head()

[Back to top](#Index:) 

### Problem 5

#### Filling the Missing Values in `name` 

**5 Points**

The `.fillna` method can accept a series and fill in the missing values based on the matching indicies.  


Use the series `ans4` you created in Problem 4 to fill in the missing values in the `name` column. 

Overwrite the earlier column and create a DataFrame with no missing values in the `name` column and assign this new DataFrame to `ans5` below.

In [ ]:
### GRADED

#ans5 = None

### BEGIN SOLUTION
#df['name'] = df['name'].fillna(df['a'].str.split('/').apply(lambda x: x[-1]))
#ans5 = df
### END SOLUTION

# Answer check
#print(type(ans5))
#ans5['name'].isnull().sum()

[Back to top](#Index:) 

### Problem 6

#### `location` missing values

**5 Points**

Note that the `location` column is still missing data.  The column is supposed to represent the location of the company as a URL in `dbpedia` or in some cases simply the name of the city the company is located in.  


Use the `isnull()` function to count how many missing values ae in the `locatio` columns. Next, use the `value_counts()` function on the `foundation` column.  Assign these counts as a series to `ans6` below.

In [ ]:
### GRADED

#ans5 = None

### BEGIN SOLUTION
#ans6 = df[df['location'].isnull()].foundation.value_counts()
### END SOLUTION

# Answer check
#print(type(ans6))
#ans6.head()

[Back to top](#Index:) 

### Problem 7

#### Replace missing `location` with `foundation`

**5 Points**

While not perfect, the values in the `foundation` column could serve as a fill in for the missing values in `location`.  Replace the missing values in `location` with their corresponding value in `foundation`.  Assign the series with the value filled for `location` to `ans7` below.

In [ ]:
### GRADED

#ans7 = None

### BEGIN SOLUTION
#ans7 = df['location'].fillna(df[df['location'].isnull()]['foundation'])
### END SOLUTION

# Answer check
#print(type(ans7))
#ans7

[Back to top](#Index:) 

### Problem 8

#### Drop rows missing revenue and profit

**5 Points**


Note that the `revenue` and `profit` columns do not have all their values.  


Use the function `dropna()` on `df` to select the non-null entries in the `subset` with columns `revenue` and `profit`.  

Assign you answer as a DataFrame to `ans8` below.

In [ ]:
### GRADED

#ans8 = None

### BEGIN SOLUTION
#ans8 = df.dropna(subset = ['revenue', 'profit'])
### END SOLUTION

# Answer check
#print(type(ans8))
#ans8.shape

[Back to top](#Index:) 

### Problem 9

#### How many rows were lost?

**5 Points**

Compare the shape of the original dataset to your solution in 8.  How many rows were lost dropping the data?  Assign your answer as an integer to `ans9` below.

In [ ]:
### GRADED

#ans9 = None

### BEGIN SOLUTION
#ans9 = df.shape[0] - df.dropna(subset = ['revenue', 'profit']).shape[0]
### END SOLUTION

# Answer check
#print(type(ans9))
#ans9

In [ ]:
from nbformat import read

# Load the Jupyter notebook file
notebook_path = 'BH_MLAI_Class_4.ipynb'
with open(notebook_path, 'r', encoding='utf-8') as file:
    notebook = read(file, as_version=4)
# Count lines of code
code_lines = 0
for cell in notebook.cells:
    if cell.cell_type == 'code':
        code_lines += len(cell.source.splitlines())

code_lines

Note that you've lost almost half of our data dropping missing revenue and profit.  Perhaps a different approach should be considered!  Take some time and examine other missing data in our dataset and some strategies for dealing with it.  If it is possible to keep observations do so.